In [19]:
# Проверка, что среда Colab активна и доступен GPU
!nvidia-smi
import torch

print("torch:", torch.__version__, "| CUDA доступна:", torch.cuda.is_available())

Sat Aug 16 15:03:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             42W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [20]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [21]:
import os

BASE = "/content/drive/MyDrive/llm-news"
for sub in ["models", "metrics", "hf_cache"]:
    os.makedirs(os.path.join(BASE, sub), exist_ok=True)

print("Созданы/проверены папки:", os.listdir(BASE))

Созданы/проверены папки: ['models', 'metrics', 'hf_cache', 'data', 'indexes', 'cache']


In [22]:
import subprocess
import sys

REPO_URL = "https://github.com/mdayssi/llm-news-summarizer-ru.git"
REPO_DIR = "/content/llm-news"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    print("Репозиторий уже есть:", REPO_DIR)

# Дальше будем работать из корня репо
%cd {REPO_DIR}
!git rev-parse --short HEAD

Репозиторий уже есть: /content/llm-news
/content/llm-news
8d2efb0


In [23]:
from pathlib import Path

env_path = Path(REPO_DIR) / ".env"
kv = {
    "EXTERNAL_MODELS_DIR": "/content/drive/MyDrive/llm-news/models",
    "EXTERNAL_METRICS_DIR": "/content/drive/MyDrive/llm-news/metrics_big",
    "EXTERNAL_CACHE_DIR": "/content/drive/MyDrive/llm-news/hf_cache",
}
text = "\n".join([f"{k}={v}" for k, v in kv.items()]) + "\n"
env_path.write_text(text, encoding="utf-8")

print(".env создано:")
print(env_path.read_text())

.env создано:
EXTERNAL_MODELS_DIR=/content/drive/MyDrive/llm-news/models
EXTERNAL_METRICS_DIR=/content/drive/MyDrive/llm-news/metrics_big
EXTERNAL_CACHE_DIR=/content/drive/MyDrive/llm-news/hf_cache



In [24]:
%pip -q install --upgrade \
  evaluate rouge-score \
  razdel bitsandbytes\
  python-dotenv pyyaml \

import datasets
import torch
import transformers
import accelerate
import rouge_score
import evaluate
import sentencepiece
import razdel
import dotenv
import yaml
import tqdm
import bitsandbytes

print("torch:", torch.__version__, "| cuda avail:", torch.cuda.is_available())
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("evaluate:", evaluate.__version__)

torch: 2.6.0+cu124 | cuda avail: True
transformers: 4.55.1
datasets: 4.0.0
evaluate: 0.4.5


In [25]:
import sys

repo_src = "/content/llm-news/src"
if repo_src not in sys.path:
    sys.path.insert(0, repo_src)
print("sys.path ok")

sys.path ok


In [26]:
2+2

4